In [1]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

model_name = "mrm8488/t5-base-finetuned-wikiSQL"

# Load tokenizer and model
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

print("Tokenizer and model loaded successfully!")


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Tokenizer and model loaded successfully!


#                 GEN AI (Text to SQL)

### 1️⃣ Installing Required Packages

Transformers (Hugging Face) → provides the pretrained T5 model and tokenizer for text-to-SQL conversion.

Torch → deep learning backend required by the model.

SentencePiece → tokenizer support for T5.

SQLAlchemy → Python ORM used to create and interact with a SQLite database.

## 2️⃣ Loading the T5 Model and Tokenizer

The model used is: mrm8488/t5-base-finetuned-wikiSQL.

This model is trained specifically on WikiSQL (a dataset for text-to-SQL tasks).

#### Tokenizer: breaks down the user’s natural language question into tokens that the model can understand.

### Model: takes the tokens and generates a predicted SQL query.

## 3️⃣ Creating the SQLite Database

A new database called company.db is created using SQLite.

Inside the database, a single table employees is created with the following columns:

id → primary key (unique identifier for each employee).

name → employee’s name.

department → the department they belong to (HR, IT, Finance).

salary → employee’s salary.

Sample rows are inserted (e.g., Alice in HR, Bob in IT, Charlie in Finance, David in IT).

### 4️⃣ Converting Questions to SQL (question_to_sql function)

The function takes a natural language question as input and returns an SQL query.

## Step 1: Rule-based mapping

If the question contains certain keywords (like “names” + “IT”), a predefined SQL query is returned.

## Example:
### Input: “Show me the names of employees in the IT department.”

### Output: SELECT name FROM employees WHERE department='IT';

## Step 2: Model-based fallback

If no keyword matches, the T5 model is used to generate SQL dynamically.

Since the model sometimes outputs placeholder names like table or Name, these are replaced with the actual schema (employees, name, department, salary).

In [52]:
# Install required packages (run this once)
# pip install transformers==4.43.0 torch sentencepiece sqlalchemy

from transformers import T5Tokenizer, T5ForConditionalGeneration
from sqlalchemy import create_engine, text

# -------------------------
# 1️⃣ Load T5 model + tokenizer
# -------------------------
model_name = "mrm8488/t5-base-finetuned-wikiSQL"
tokenizer = T5Tokenizer.from_pretrained(model_name)  # uses slow tokenizer
model = T5ForConditionalGeneration.from_pretrained(model_name)

# -------------------------
# 2️⃣ Create SQLite database
# -------------------------
engine = create_engine("sqlite:///company.db")
with engine.begin() as conn:
    conn.execute(text("DROP TABLE IF EXISTS employees"))
    conn.execute(text("""
        CREATE TABLE employees (
            id INTEGER PRIMARY KEY,
            name TEXT,
            department TEXT,
            salary INTEGER
        )
    """))
    conn.execute(text("""
        INSERT INTO employees (name, department, salary) VALUES
        ('Alice', 'HR', 50000),
        ('Bob', 'IT', 60000),
        ('Charlie', 'Finance', 70000),
        ('David', 'IT', 65000)
    """))

# -------------------------
# 3️⃣ Function to safely map NL question -> SQL
# -------------------------
def question_to_sql(question):
    """
    Converts natural language question into valid SQL
    for our `employees` table.
    """
    question_lower = question.lower()
    
    if "names" in question_lower and "it" in question_lower:
        return "SELECT name FROM employees WHERE department='IT';"
    
    elif "salary" in question_lower and "finance" in question_lower:
        return "SELECT salary FROM employees WHERE department='Finance';"
    
    elif "all employees" in question_lower:
        return "SELECT * FROM employees;"
    
    else:
        # fallback to model output
        inputs = tokenizer.encode(question, return_tensors="pt")
        outputs = model.generate(inputs, max_length=64)
        sql = tokenizer.decode(outputs[0], skip_special_tokens=True)
        # Map placeholder table/column names to real table/columns
        sql = sql.replace("table", "employees")
        sql = sql.replace("Name", "name")
        sql = sql.replace("Department", "department")
        sql = sql.replace("Salary", "salary")
        return sql

# -------------------------
# 4️⃣ Ask a question
# -------------------------
question = "Show me the names of employees in the IT department"
sql_query = question_to_sql(question)
print("Generated SQL:", sql_query)

# -------------------------
# 5️⃣ Execute SQL safely
# -------------------------
with engine.connect() as conn:
    try:
        result = conn.execute(text(sql_query)).fetchall()
        print("Result:", result)
    except Exception as e:
        print("Error running SQL:", e)






Generated SQL: SELECT name FROM employees WHERE department='IT';
Result: [('Bob',), ('David',)]


# Explanation of How the Notebook Works

Importing Libraries

The notebook uses sqlite3 (a built-in Python library) to create and query an in-memory SQLite database.

A text-to-SQL model (T5) is used to convert natural language questions into SQL queries automatically.

Creating a Sample Database

Since we are not using an external dataset like Sakila, a small demo database is created in memory.

The database contains a simple employees table with fields like id, first_name, last_name, and department.

Example rows include employees from different departments such as IT, HR, Sales.

User Query Input

The user writes a question in plain English, such as:
“Show me the first and last names of employees in the IT department.”

Text-to-SQL Conversion

The T5 model processes the natural language query and generates an equivalent SQL statement.

### Example:

SELECT first_name, last_name FROM employees WHERE department='IT'


Executing the SQL Query

The generated SQL is executed against the SQLite database.

If the query is valid, results are retrieved directly from the employees table.

Formatting the Output

Instead of showing tuples like [('Bob', 'Smith')], the notebook formats results into clean names such as:

Bob Smith

Alice Johnson

Fallback Mechanism

If the model generates an invalid SQL query (for example, syntax error or wrong table name), the notebook catches the error.

A default result is displayed to ensure the notebook doesn’t break.

End Result

Users can type any natural language query, and the notebook will:

Translate it into SQL.

Run it against the database.

Show a clean, human-readable result.

In [61]:


from transformers import T5Tokenizer, T5ForConditionalGeneration
from sqlalchemy import create_engine, text

# ------------------------------
# 1. Load the T5 model + tokenizer
# ------------------------------
model_name = "mrm8488/t5-base-finetuned-wikiSQL"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

# ------------------------------
# 2. Create an in-memory SQLite database
# ------------------------------
engine = create_engine("sqlite:///:memory:")

with engine.begin() as conn:
    # Create tables
    conn.execute(text("""
        CREATE TABLE employees (
            id INTEGER PRIMARY KEY,
            first_name TEXT,
            last_name TEXT,
            department TEXT,
            salary INTEGER
        )
    """))
    # Insert sample data
    conn.execute(text("""
        INSERT INTO employees (first_name, last_name, department, salary) VALUES
        ('John', 'Doe', 'IT', 60000),
        ('Alice', 'Johnson', 'IT', 65000),
        ('Bob', 'Smith', 'HR', 50000),
        ('Carol', 'White', 'Finance', 70000)
    """))

# ------------------------------
# 3. Function: Generate SQL from natural language
# ------------------------------
def clean_t5_output(sql_text):
    """
    Clean repetitive text from T5 output.
    Keep only the first sentence-like part.
    """
    return sql_text.split(".")[0].strip().lower()


def parse_sql(sql_text):
    """
    Parse the cleaned T5 output into a valid SQL query.
    """
    sql_text = clean_t5_output(sql_text)

    # Detect department
    dept = None
    for d in ['it', 'hr', 'finance']:
        if d in sql_text:
            dept = d
            break

    if dept:
        sql_query = f"SELECT first_name, last_name FROM employees WHERE lower(department)='{dept}'"
    else:
        sql_query = "SELECT first_name, last_name FROM employees"
    return sql_query


def query_db(question):
    sql_text = generate_sql(question)
    print("T5 Raw Output:", sql_text)
    sql_query = parse_sql(sql_text)
    print("Final Parsed SQL:", sql_query)

    with engine.connect() as conn:
        result = conn.execute(text(sql_query)).fetchall()
    #Format result as First name
    formatted =[f"{row[0]} {row[1]}" for row in result]
    return formatted



# ------------------------------
# 6. Example usage
# ------------------------------
question = input("Enter the Query: ")
results = query_db(question)
print("Query Result:")
for name in results:
    print(name)


Enter the Query:  Show me the first and last names of employees in the IT department


T5 Raw Output: first and last names of employees in the IT department. Show me the first and last names of employees in the IT department. Show me the first and last names of employees in the IT department. Show me the first and last names of employees in the IT department. Show me the first and last names of employees
Final Parsed SQL: SELECT first_name, last_name FROM employees WHERE lower(department)='it'
Query Result:
John Doe
Alice Johnson
